# Named Entity Recognition for the decicontas.br dataset

In [144]:
import re
import ast
import json

import pandas as pd

from typing import List, Tuple
from langchain_openai import  AzureChatOpenAI
from langchain.prompts import PromptTemplate
from langchain_core.prompts import StringPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from sklearn.metrics import precision_score, recall_score, f1_score
from tools.schema import (
    Entidade, 
    NERResposta
)
from dotenv import load_dotenv

load_dotenv()

llm_azure_gpt_4o = AzureChatOpenAI(model_name="gpt-4o")

In [117]:
df = pd.read_json("dataset/labeled_data/fewshot_decicontas.json")
df_fewshot = df[['annotations', 'data']]

EXEMPLOS = """
"""

for _, row in df_fewshot.iterrows():
    texto = row['data']['text']
    entidades = json.dumps([x['value'] for x in row['annotations'][0]['result']])
    EXEMPLOS += f"Texto: {texto}\nEntidades extraídas: {entidades}\n"
EXEMPLOS += "Texto: {input}\nEntidades extraídas: \n"

In [118]:
print(EXEMPLOS)


Texto:                                 DECIDEM os Conselheiros do Tribunal de Contas do Estado, à unanimidade, em consonância com a informação do Corpo Técnico e com o parecer do Ministério Público que atua junto a esta Corte de Contas, acolhendo integralmente o voto do Conselheiro Relator, julgar: a) pela DENEGAÇÃO DE REGISTRO ao ato concessivo da aposentadoria e à despesa dele decorrente; b) pela determinação ao IPERN, à vista da Lei Complementar Estadual nº 547/2015, para que, no prazo de 60 (sessenta) dias, após o trânsito em julgado desta decisão, adote as correções necessárias para regularização do ato concessório, do cálculo dos proventos e de sua respectiva implantação; c) no caso de descumprimento da presente decisão, a responsabilização do titular da pasta responsável por seu atendimento, sem prejuízo da multa cominatória desde já fixada no valor de R$ 50,00 (cinquenta reais) por dia que superar o interregno fixado no item `b`, com base no art. 110 da Lei Complementar Estadu

# Testing with golden labels

In [76]:
df_golden = pd.read_json("dataset/labeled_data/fewshot_decicontas.json")

In [98]:
texts = []
ner_labels = []

for _, row in df.iterrows():
    texto = row['data']['text']
    anotacoes = json.dumps([x['value'] for x in row['annotations'][0]['result']])

    texts.append(texto)
    ner_labels.append(anotacoes)

In [145]:
# Output parser simples de string
output_parser = StrOutputParser()

# Cadeia como função com str.format()
def run_chain(texto_input: str) -> List[dict]:
    prompt = EXEMPLOS.replace("{input}", texto_input)
    resposta = output_parser.invoke(llm_azure_gpt_4o.invoke(prompt)).strip()

    # Remove blocos markdown e limpa
    resposta = re.sub(r"```json|```", "", resposta).strip()

    try:
        return ast.literal_eval(resposta)
    except Exception as e:
        print(f"[Erro parseando resposta]:\n{resposta}\n-> {e}")
        return []

In [151]:
def extract_triples(entity_list: List[dict]) -> List[Tuple[int, int, str]]:
    triples = []
    for e in entity_list:
        label = e["labels"]
        if isinstance(label, list):
            label = label[0]
        triples.append((e["start"], e["end"], label))
    return triples

def evaluate_ner_llm(texts, golden_labels):
    y_true, y_pred = [], []

    for i, (text, gold_str) in enumerate(zip(texts, golden_labels)):
        try:
            gold = ast.literal_eval(gold_str)
            gold_triples = extract_triples(gold)

            pred = run_chain(text)
            pred_triples = extract_triples(pred)

            gold_set = set(gold_triples)
            pred_set = set(pred_triples)

            for ent in pred_set:
                y_pred.append(1 if ent in gold_set else 0)
            for ent in gold_set:
                y_true.append(1 if ent in pred_set else 0)

        except Exception as e:
            print(f"[Erro na instância {i}] {e}")

    return {
        "precision": precision_score(y_true, y_pred, zero_division=0),
        "recall": recall_score(y_true, y_pred, zero_division=0),
        "f1": f1_score(y_true, y_pred, zero_division=0),
        "total_gold": sum(y_true),
        "total_pred": sum(y_pred),
        "samples": len(texts)
    }

In [132]:
results = evaluate_ner_llm(texts, ner_labels)
print(results)

[Erro na instância 0] 'label'
[Erro na instância 1] '"start"'
[Erro na instância 2] 'label'
[Erro na instância 3] 'label'
[Erro na instância 4] '"start"'
[Erro na instância 5] 'label'
[Erro na instância 6] 'label'
[Erro na instância 7] 'label'
[Erro na instância 8] 'label'
[Erro na instância 9] 'label'
[Erro na instância 10] 'label'
[Erro na instância 11] '"start"'
{'precision': 0.0, 'recall': 0.0, 'f1': 0.0, 'total_gold': 0, 'total_pred': 0, 'samples': 12}


In [152]:
y_true, y_pred = [], []

for i, (text, gold_str) in enumerate(zip(texts, ner_labels)):

    gold = ast.literal_eval(gold_str)
    gold_triples = extract_triples(gold)

    pred = run_chain(text)
    pred_triples = extract_triples(pred)

    gold_set = set(gold_triples)
    pred_set = set(pred_triples)

    for ent in pred_set:
        y_pred.append(1 if ent in gold_set else 0)
    for ent in gold_set:
        y_true.append(1 if ent in pred_set else 0)

    break
